In [ ]:
# Install conda (to install rdkit)
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# Now install rdkit. Can take a bit.
!mamba install -c conda-forge rdkit


In [ ]:
# Import rdkit

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import Crippen
from rdkit.Chem import Lipinski
import pandas as pd

In [ ]:
# You can also load molecules from a file in other formats (e.g., SDF). Note
# that SDF files can contain multiple molecules.

# Download mol.sdf
import requests
url = 'https://raw.githubusercontent.com/durrantlab/colab-support-files/refs/heads/main/mols.sdf'
response = requests.get(url)
sdf_contents = response.text
with open("mols.sdf", "w") as f:
    f.write(sdf_contents)

# Load the SDF file
suppl = Chem.SDMolSupplier('mols.sdf')
suppl

In [ ]:
# If any molecules can't be read, they will be None. Filter them out.
ms = [x for x in suppl if x is not None]

# How many molecules did we get?
print(len(ms))

# Cheminformatics #1

In [ ]:
# Show the first molecule. In this case, the molecules have 3D coordinates
# assigned.
ms[0]

In [ ]:
# To improve the drawing, let's calculate 2D coordiantes instead.
for m in ms:
    AllChem.Compute2DCoords(m)

ms[0]

In [ ]:
# You can also use rdkit to create mol objects from smiles strings

housane = Chem.MolFromSmiles('C1CC2C1C2')
housane

In [ ]:
# RDKIT represents molecules as objects. There are many ways of creating these
# molecule objects. For example, you can create them from SMILES strings:
kid = Chem.MolFromSmiles('O1CCOC1c1c(C#CC(C)(C)C)cc(c(C#CC(C)(C)C)c1)C#Cc1cc(C#CCCC)cc(C#CCCC)c1')
kid

# Cheminformatics #2


## Similarity calculations

In [ ]:
# You can use RDKit to measure the chemical similarity between molecules.
# Let's first consider two molecules that are very similar, estrogen and
# testosterone.
estrogen_smiles = "C[C@]12CC[C@H]3[C@H]([C@@H]1C[C@H]([C@@H]2O)O)CCC4=C3C=CC(=C4)O"
estrogen = Chem.MolFromSmiles(estrogen_smiles)
estrogen

In [ ]:
testosterone_smiles = "C[C@]12CC[C@H]3[C@H]([C@@H]1CC[C@@H]2O)CCC4=CC(=O)CC[C@]34C"
testosterone = Chem.MolFromSmiles(testosterone_smiles)
testosterone

In [ ]:
# Let's calculate the MACCS fingerprint for each of these molecules.
estrogen_maccs = AllChem.GetMACCSKeysFingerprint(estrogen)
testosterone_maccs = AllChem.GetMACCSKeysFingerprint(testosterone)

# Let's look at the values in estrogen_maccs (bits)
print(estrogen_maccs.ToBitString())

In [ ]:
# Now let's calculdate the Tanimoto (Jaccard) similarity between these MACCS
# fingerprints
DataStructs.FingerprintSimilarity(estrogen_maccs, testosterone_maccs)

In [ ]:
# Now let's load a molecule that is very different.
thyroxine_smiles = "C1=C(C=C(C(=C1I)OC2=CC(=C(C(=C2)I)O)I)I)C[C@@H](C(=O)O)N"
thyroxine = Chem.MolFromSmiles(thyroxine_smiles)
thyroxine

In [ ]:
# We will similarly calculate it's MACCS fingerprint and Tanimoto similarity to
# estrogen. The similarity is lower because the molecule is not very similar
# to estrogen.
thyroxine_maccs = AllChem.GetMACCSKeysFingerprint(thyroxine)
DataStructs.FingerprintSimilarity(estrogen_maccs, thyroxine_maccs)

## Butina to create a diversity set

In [ ]:
# Use the Butina algorithm to cluster the compounds in the ms list. See
# https://projects.volkamerlab.org/teachopencadd/talktorials/T005_compound_clustering.html

# NOTE: I won't ask you how to run Butina using RDKit specifically. Good to
# understand how Butina algorithm works generally, though.
# Convert all the molecules to fingerprints
fps = [Chem.RDKFingerprint(m) for m in ms]
def tanimoto_distance_matrix(fp_list):
  """Calculate distance matrix for fingerprint list"""
  dissimilarity_matrix = []
  # Notice how we are deliberately skipping the first and last items in the
  # list because we don't need to compare them against themselves
  for i in range(1, len(fp_list)):
    # Compare the current fingerprint against all the previous ones in the
    # list
    similarities = DataStructs.BulkTanimotoSimilarity(fp_list[i], fp_list[:i])
    # Since we need a distance matrix, calculate 1-x for every element in
    # similarity matrix
    dissimilarity_matrix.extend([1 - x for x in similarities])
  return dissimilarity_matrix
distance_matrix = tanimoto_distance_matrix(fps)

In [ ]:
# Run the Butina algorithm to cluster the molecules. NOTE: I won't ask you how
# to run Butina using RDKit specifically. Good to understand how Butina
# algorithm works generally, though.
clusters = Butina.ClusterData(distance_matrix, len(fps), 0.125, isDistData=True)
clusters = sorted(clusters, key=len, reverse=True)

In [ ]:
# `clusters` is a list of tuples, where each tuple contains the indices of the
# molecules in that cluster.
print(len(clusters))
clusters

In [ ]:
# Let's look at the first cluster
first_cluster = clusters[0]
second_cluster = clusters[1]
first_cluster

In [ ]:
# Look at the structures in the first cluster. RDKit can display molecules in a
# grid.
for i in first_cluster:
  AllChem.Compute2DCoords(ms[i])

Draw.MolsToGridImage([ms[i] for i in first_cluster], molsPerRow=3, useSVG=True, subImgSize=(300,300))

In [ ]:
# Look at the structures in the second cluster, too.
for i in second_cluster:
  AllChem.Compute2DCoords(ms[i])

Draw.MolsToGridImage([ms[i] for i in second_cluster], molsPerRow=3, useSVG=True, subImgSize=(300,300))

## RDKit can also construct similarity libraries based on common substructures

In [ ]:
# You can also use rdkit to search for molecules with certain substructures.
# Define the substructure.

substruct = Chem.MolFromSmiles('c1cccc([OH])c1OC')
substruct

In [ ]:
# Search throguh all the molecules to find matches
matches = []
nonmatches = []
for m in ms:
    if m.HasSubstructMatch(substruct):
        matches.append(m)
    else:
        nonmatches.append(m)

# How many matches are there?
print(len(matches))

# Let's see them
for m in matches:
  AllChem.Compute2DCoords(m)
Draw.MolsToGridImage(matches, molsPerRow=3, useSVG=True, subImgSize=(400,400))



In [ ]:
# These matches are similar to your query molecule (substruct) and so are all
# similar to each other. An example of a tageted library.

## You can also use RDKit to create a similarity set

In [ ]:
# Tanimoto similarity acts on fingerprints, so lets calculate fingerprints for
# each molecule in ms.

fps = [Chem.RDKFingerprint(m) for m in ms]

In [ ]:
# Now let's get a molecule to search for.
query = Chem.MolFromSmiles('O=C1NC=CC(=O)N1')
query_fp = Chem.RDKFingerprint(query)  # Note the use of RDKfingerprint, not MACCS
query

In [ ]:
# Calculate the tanimoto similarity between the query and each molecule in ms.

tanimoto = [
    DataStructs.FingerprintSimilarity(query_fp, fp, metric=DataStructs.TanimotoSimilarity)
    for fp in fps
]
tanimoto[:10]

In [ ]:
# Make a list of (tanimoto, molecule) pairs.
tanimoto_mol_pairs = list(zip(tanimoto, ms))

# Sort by the tanimoto score (first item in pair) from largest to smallest.
tanimoto_mol_pairs = sorted(tanimoto_mol_pairs, key=lambda x: x[0], reverse=True)

# Just show the top 10.
tanimoto_mol_pairs[:10]

In [ ]:
# What is the most similar molecule?
top_score_and_mol = tanimoto_mol_pairs[0]
top_mol = top_score_and_mol[1]
top_mol

In [ ]:
# Remember the beauty of 2D coordinates. Necessary because the sdf file
# I loaded had 3D coordinates assigned.
AllChem.Compute2DCoords(top_mol)
top_mol

In [ ]:
# What is the least similar?
worst_score_and_mol = tanimoto_mol_pairs[-1]
worst_mol = worst_score_and_mol[1]
AllChem.Compute2DCoords(worst_mol)
worst_mol

In [ ]:
# The similarity set could be defined as the set of N most similar compounds
similarity_set = tanimoto_mol_pairs[:10]
similar_mols = [m[1] for m in similarity_set]
for m in similar_mols:
  AllChem.Compute2DCoords(m)
Draw.MolsToGridImage(similar_mols, molsPerRow=3, useSVG=True, subImgSize=(300,300))



## Using RDKit to calculate molecular properties

In [ ]:
# Calculate the molecular properties (logP, MW, number of hydrogen bond donors,

# Create a DataFrame
df = pd.DataFrame(columns=["SMILES", "MW", "logP", "HBD", "HBA"])

# Populate the DataFrame
for i, m in enumerate(ms):
    df.loc[i] = [
        Chem.MolToSmiles(ms[i]),
        round(AllChem.CalcExactMolWt(m), 2),
        round(Crippen.MolLogP(m), 2),
        round(Lipinski.NumHDonors(m), 2),
        round(Lipinski.NumHAcceptors(m), 2),
    ]
df